# Loading Responses

In [19]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("gaze-analysis/result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "gemini-1.5-pro" & df$list_id != -1 & df$Part != "p0", ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))
df$Proximity <- df$Proximity - mean(df$Proximity)
df$n_candidates <- df$n_candidates - mean(df$n_candidates)

Loading required package: car

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘car’”


also installing the dependencies ‘stringi’, ‘purrr’, ‘stringr’, ‘tidyr’, ‘cowplot’, ‘Deriv’, ‘modelr’, ‘microbenchmark’, ‘broom’, ‘dplyr’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘survival’, ‘carData’, ‘abind’, ‘nnet’, ‘pbkrtest’, ‘quantreg’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Loading required package: carData

Rows: 156780 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Fitting models

In [5]:
model0 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Actor:Candidates) + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0488344 (tol = 0.002, component 1)”


In [6]:
model1 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

In [7]:
model2 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Actor:Candidates) + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

In [8]:
model3 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

In [13]:
model3_5 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Actor:Candidates), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

In [15]:
model4 <- glm(Accuracy ~ Angle * Proximity * n_candidates * Actor, 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

In [36]:
model5 <- glmer(Accuracy ~ Angle + Proximity * Actor + n_candidates + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

In [37]:
summary(model5)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + Proximity * Actor + n_candidates + (1 | Stimulus_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  7014.5   7071.3  -3499.3   6998.5     8992 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.9918 -0.1742 -0.1041  0.1705  3.1080 

Random effects:
 Groups      Name        Variance Std.Dev.
 Stimulus_ID (Intercept) 16.19    4.024   
Number of obs: 9000, groups:  Stimulus_ID, 900

Fixed effects:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)        0.5490     0.3026   1.814   0.0696 .  
Angleleft         -0.4454     0.3795  -1.174   0.2405    
Angleright        -0.2559     0.3807  -0.672   0.5014    
Proximity         -1.0186     0.2608  -3.905 9.43e-05 ***
ActorY            -0.1653     0.3131  -0.528   0.5976    
n_candidates      -0.9007     0.2067  -4.357 1.32e-05 ***


# Model selection

In [ ]:
anova(model1, model2)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model1,8,7011.277,7068.117,-3497.639,6995.277,NA,NA,NA
model2,8,7017.500,7074.339,-3500.750,7001.500,0,0,NA


In [9]:
anova(model1, model3)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model3,7,7015.499,7065.234,-3500.750,7001.499,NA,NA,NA
model1,8,7011.277,7068.117,-3497.639,6995.277,6.222326,1,0.01261493


In [14]:
anova(model3, model3_5)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model3,7,7015.499,7065.234,-3500.750,7001.499,NA,NA,NA
model3_5,7,12024.321,12074.056,-6005.161,12010.321,0,0,NA


In [16]:
anova(model3, model4)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model3,7,7015.499,7065.234,-3500.750,7001.499,NA,NA,NA
model4,24,12007.389,12177.908,-5979.694,11959.389,0,17,1


In [38]:
anova(model1, model5)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model1,8,7011.277,7068.117,-3497.639,6995.277,NA,NA,NA
model5,8,7014.502,7071.342,-3499.251,6998.502,0,0,NA


In [ ]:
model <- model1

# Interpretation

In [ ]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + Proximity + n_candidates + Actor + (1 | Stimulus_ID) +  
    (1 | Prompt_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  7011.3   7068.1  -3497.6   6995.3     8992 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1910 -0.1774 -0.0992  0.1699  3.1835 

Random effects:
 Groups      Name        Variance Std.Dev.
 Stimulus_ID (Intercept) 16.34973 4.043   
 Prompt_ID   (Intercept)  0.02464 0.157   
Number of obs: 9000, groups:  Stimulus_ID, 900; Prompt_ID, 12

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)    0.5441     0.3072   1.771 0.076559 .  
Angleleft     -0.4442     0.3813  -1.165 0.244009    
Angleright    -0.2524     0.3825  -0.660 0.509341    
Proximity     -0.7219     0.1930  -3.741 0.000184 ***
n_candidates  -0.9059     0.2077  -4.362 1.29e-05 ***


In [ ]:
log_odds_means <- emmeans(model, spec = "Angle")
print(log_odds_means)
pairs(log_odds_means)
log_odds_means <- emmeans(model, spec = "Angle", regrid = "response")
print(log_odds_means)
pairs(log_odds_means)
               
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

 Angle emmean    SE  df asymp.LCL asymp.UCL
 front  0.111 0.276 Inf    -0.430     0.651
 left  -0.333 0.272 Inf    -0.866     0.200
 right -0.142 0.273 Inf    -0.677     0.394

Results are averaged over the levels of: Actor 
Results are given on the logit (not the response) scale. 
Confidence level used: 0.95 


 contrast      estimate    SE  df z.ratio p.value
 front - left     0.444 0.381 Inf   1.165  0.4742
 front - right    0.252 0.382 Inf   0.660  0.7868
 left - right    -0.192 0.380 Inf  -0.505  0.8689

Results are averaged over the levels of: Actor 
Results are given on the log odds ratio (not the response) scale. 
P value adjustment: tukey method for comparing a family of 3 estimates 

 Angle  prob     SE  df asymp.LCL asymp.UCL
 front 0.528 0.0686 Inf     0.393     0.662
 left  0.418 0.0660 Inf     0.288     0.547
 right 0.465 0.0678 Inf     0.332     0.598

Results are averaged over the levels of: Actor 
Confidence level used: 0.95 


 contrast      estimate     SE  df z.ratio p.value
 front - left    0.1101 0.0938 Inf   1.174  0.4688
 front - right   0.0629 0.0951 Inf   0.662  0.7858
 left - right   -0.0472 0.0933 Inf  -0.506  0.8686

Results are averaged over the levels of: Actor 
P value adjustment: tukey method for comparing a family of 3 estimates 

term,Estimate,Std. Error,z value,Pr(>|z|),effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.5441380,0.3072468,1.7710128,7.655858e-02,4.434438,0.12270733
Angleleft,-0.4442305,0.3813060,-1.1650237,2.440094e-01,4.434438,-0.10017742
Angleright,-0.2523914,0.3824899,-0.6598642,5.093410e-01,4.434438,-0.05691622
Proximity,-0.7219165,0.1929979,-3.7405405,1.836249e-04,4.434438,-0.16279775
n_candidates,-0.9059379,0.2076662,-4.3624708,1.286017e-05,4.434438,-0.20429602
ActorY,-0.1659441,0.3146023,-0.5274727,5.978654e-01,4.434438,-0.03742169
